<a href="https://colab.research.google.com/github/anitamezzetti/ML_finance/blob/main/generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import os
from time import time
import pandas as pd

In [28]:
from ipynb.fs.full.closed_form_solution import f, p, p1, p2, call_price

In [29]:
start = time()

# Set the constants
# -----------------------------------------------------------------------------------
r = 0
k = 2
V0 = 0.010
sigma = 0.61
theta = 0.019
kappa = 6.21

T1 = 10

rho = -0.5

In [30]:
S0 = np.arange(1,4,0.01)

Stock prices simulation:

In [31]:
def stock_price_generator (t_max, n ,m, r, S0, k, V0, sigma, theta, kappa, rho):
    dt = t_max / n
    
    # Brownian motions:
    dw_v = np.random.normal(size=(m, n)) * np.sqrt(dt)
    dw_i = np.random.normal(size=(m, n)) * np.sqrt(dt)

    dw_s = rho * dw_v + np.sqrt(1.0 - rho ** 2) * dw_i

    # Perform time evolution 
    s = np.empty((m, n + 1)) # initialisation stock prices vector
    s[:, 0] = S0

    v = np.ones(m) * V0

    for t in range(n):
        dv = kappa * (theta - v) * dt + sigma * np.sqrt(v) * dw_v[:, t]
        ds = r * s[:, t] * dt + np.sqrt(v) * s[:, t] * dw_s[:, t]

        v = np.clip(v + dv, a_min=0.0, a_max=None)
        s[:, t + 1] = s[:, t] + ds
      
        
    return s
    

In [32]:
def find_expected_payoff(stock_path, k, r, t_max):
    payoff = max(stock_path[-1] - k, 0) # one payoff for each simulation
    c = payoff * np.exp(-r * t_max)     # in case r=0, this step is useless
    
    return c

In [33]:
time_maturity = 3
num_simulations = 100
n = 100


In [34]:
df = pd.DataFrame(columns=['price', 'stock_zero'])

In [35]:
# fill the dataset
for s0 in S0:
    s = stock_price_generator (time_maturity, n, num_simulations, r, s0, k, V0, sigma, theta, kappa, rho)

    for stock_path in s:
        p = find_expected_payoff(stock_path, k, r, time_maturity)
        
        new_row = {
                      'price':p, 
                      'stock_zero':stock_path[T1],
                  }
        #append row to the dataframe
        df = df.append(new_row, ignore_index=True)


s0 1.0
s0 1.01
s0 1.02
s0 1.03
s0 1.04
s0 1.05
s0 1.06
s0 1.07
s0 1.08
s0 1.09
s0 1.1
s0 1.11
s0 1.12
s0 1.1300000000000001
s0 1.1400000000000001
s0 1.1500000000000001
s0 1.1600000000000001
s0 1.1700000000000002
s0 1.1800000000000002
s0 1.1900000000000002
s0 1.2000000000000002
s0 1.2100000000000002
s0 1.2200000000000002
s0 1.2300000000000002
s0 1.2400000000000002
s0 1.2500000000000002
s0 1.2600000000000002
s0 1.2700000000000002
s0 1.2800000000000002
s0 1.2900000000000003
s0 1.3000000000000003
s0 1.3100000000000003
s0 1.3200000000000003
s0 1.3300000000000003
s0 1.3400000000000003
s0 1.3500000000000003
s0 1.3600000000000003
s0 1.3700000000000003
s0 1.3800000000000003
s0 1.3900000000000003
s0 1.4000000000000004
s0 1.4100000000000004
s0 1.4200000000000004
s0 1.4300000000000004
s0 1.4400000000000004
s0 1.4500000000000004
s0 1.4600000000000004
s0 1.4700000000000004
s0 1.4800000000000004
s0 1.4900000000000004
s0 1.5000000000000004
s0 1.5100000000000005
s0 1.5200000000000005
s0 1.5300000000000

In [26]:
df.groupby('stock_zero').mean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms


In [18]:
closed_price = []
for s0 in S0:
    cp = call_price(kappa, theta, sigma, rho, V0 ,r , time_maturity ,s0, k)
    closed_price.append(cp)

C:\Users\anita\Desktop\ML finance\ML_finance\closed_form_solution.ipynb:46: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  "    d = cmath.sqrt((rho * sigma * phi * 1j - b)**2 - sigma**2 * (2 * u * phi * 1j - phi**2))\n",


In [19]:
import seaborn as sns
sns.scatterplot(data=df, x="stock", y="price", alpha =0.5)
plt.plot(S0, closed_price, color='r')
plt.show()

ValueError: Could not interpret value `stock` for parameter `x`

In [20]:
x, y = np.random.random((2, 100))*2
fig, ax = plt.subplots()
ax.scatter(x, y, c='black')
line = mlines.Line2D([0, 1], [0, 1], color='red')
transform = ax.transAxes
line.set_transform(transform)
ax.add_line(line)
plt.show()

NameError: name 'plt' is not defined